In [24]:
import pandas as pd
import numpy as np

# Predicting 2018-19 winners and teams that have relegated using linear regression
# model from previous year data

# This model cannot be used in real world as teams performace depends upon the players and managers and
# not on previous year datas

In [25]:
data = pd.read_csv("/content/drive/MyDrive/Project_reviews/FMEL_Dataset.csv")
data.head(10)

,id,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,timestamp
0,1,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970,21938400
1,2,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970,21938400
2,3,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970,21938400
3,4,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970,22024800
4,5,1970-71,1,1,Elche,Granada,1,1,13/09/1970,22024800
5,6,1970-71,1,1,Espanol,Sevilla,0,1,13/09/1970,22024800
6,7,1970-71,1,1,Sabadell,Real Sociedad,0,0,13/09/1970,22024800
7,8,1970-71,1,1,Zaragoza,CD Malaga,0,0,13/09/1970,22024800
8,9,1970-71,1,2,Valencia,Las Palmas,5,1,19/09/1970,22543200
9,10,1970-71,1,2,Atletico de Madrid,Atletico de Bilbao,2,0,20/09/1970,22629600


In [26]:
data = data.drop(["id","timestamp"],axis=1)
data.head(5)

,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date
0,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970
1,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970
2,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970
3,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970
4,1970-71,1,1,Elche,Granada,1,1,13/09/1970


In [27]:
condition = [(data["localGoals"] > data["visitorGoals"]),(data["localGoals"] < data["visitorGoals"]),
             (data["localGoals"] == data["visitorGoals"])]
value = ["local","visitor","draw"]

data["result"] = np.select(condition,value)

data.head(5)

,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,result
0,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,12/09/1970,draw
1,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,12/09/1970,draw
2,1970-71,1,1,Real Madrid,Valencia,2,0,12/09/1970,local
3,1970-71,1,1,Celta de Vigo,Sporting de Gijon,2,0,13/09/1970,local
4,1970-71,1,1,Elche,Granada,1,1,13/09/1970,draw


In [28]:
df = data.merge(pd.get_dummies(data['result']), left_index=True, right_index=True)

df['local_wins_this_season'] = df.groupby(['season','localTeam'])['local'].transform('sum')
df['visitor_wins_this_season'] = df.groupby(['season','visitorTeam'])['visitor'].transform('sum')

print(" ")

In [29]:
visitor_results = (df
                   .groupby(['season', 'visitorTeam'])['visitor']
                   .sum()
                   .reset_index()
                   .rename(columns={'visitorTeam': 'team',
                                    'visitor': 'visitor_wins'}))

local_results = (df
                 .groupby(['season', 'localTeam'])['local']
                 .sum()
                 .reset_index()
                 .rename(columns={'localTeam': 'team',
                                  'local': 'local_wins'}))

wins_per_season = visitor_results.merge(local_results, on=['season', 'team'])

wins_per_season['total_wins'] = wins_per_season['visitor_wins'] + wins_per_season['local_wins']

wins_per_season.head(5)

,season,team,visitor_wins,local_wins,total_wins
0,1970-71,Atletico de Bilbao,3,11,14
1,1970-71,Atletico de Madrid,4,13,17
2,1970-71,Barcelona,8,11,19
3,1970-71,Betis,5,15,20
4,1970-71,Burgos,3,16,19


In [30]:
total_wins_sorted_desc = (wins_per_season
                          .groupby(['team'])['total_wins']
                          .sum()
                          .sort_values(ascending=False)
                          .reset_index()['team'])

wins_per_season_pivot = (wins_per_season
                         .pivot_table(index='team',
                                      columns='season',
                                      values='total_wins')
                         .fillna(0)
                         .reindex(total_wins_sorted_desc))

In [31]:
# total wins of each teams for each season

print(wins_per_season_pivot)

season                     1970-71  1971-72  1972-73  1973-74  1974-75  \
team                                                                     
Real Madrid                   17.0     19.0     17.0     13.0     20.0   
Barcelona                     19.0     17.0     18.0     21.0     15.0   
Atletico de Madrid            17.0     14.0     20.0     18.0     11.0   
Valencia                      18.0     19.0     12.0     13.0     12.0   
Sevilla                       13.0      9.0     16.0     15.0     22.0   
...                            ...      ...      ...      ...      ...   
Ontinyent                      8.0      0.0      0.0      0.0      0.0   
Universidad de Las Palmas      0.0      0.0      0.0      0.0      0.0   
Lorca FC                       0.0      0.0      0.0      0.0      0.0   
Moscardo                       6.0      0.0      0.0      0.0      0.0   
Mollerussa                     0.0      0.0      0.0      0.0      0.0   

season                     1975-76  1

In [32]:
df = pd.DataFrame(wins_per_season_pivot.to_records())

df

,team,1970-71,1971-72,1972-73,1973-74,1974-75,1975-76,1976-77,1977-78,1978-79,...,2008-09,2009-10,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18
0,Real Madrid,17.0,19.0,17.0,13.0,20.0,20.0,12.0,22.0,16.0,...,25.0,31.0,29.0,32.0,26.0,27.0,30.0,28.0,29.0,22.0
1,Barcelona,19.0,17.0,18.0,21.0,15.0,18.0,18.0,16.0,16.0,...,27.0,31.0,30.0,28.0,32.0,27.0,30.0,29.0,28.0,28.0
2,Atletico de Madrid,17.0,14.0,20.0,18.0,11.0,18.0,19.0,16.0,14.0,...,20.0,13.0,17.0,15.0,23.0,28.0,23.0,28.0,23.0,23.0
3,Valencia,18.0,19.0,12.0,13.0,12.0,12.0,13.0,16.0,14.0,...,18.0,21.0,21.0,17.0,19.0,13.0,22.0,11.0,13.0,22.0
4,Sevilla,13.0,9.0,16.0,15.0,22.0,13.0,11.0,13.0,12.0,...,21.0,19.0,17.0,13.0,14.0,18.0,23.0,14.0,21.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Ontinyent,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,Universidad de Las Palmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,Lorca FC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
113,Moscardo,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
col = [i for i in range(1970,2018)]
col_ind = [j for j in range(1,49)]
print(len(col))
print(len(col_ind))

48
48


In [34]:
old_names = df.columns[col_ind]
df.rename(columns=dict(zip(old_names, col)), inplace=True)
df

,team,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Real Madrid,17.0,19.0,17.0,13.0,20.0,20.0,12.0,22.0,16.0,...,25.0,31.0,29.0,32.0,26.0,27.0,30.0,28.0,29.0,22.0
1,Barcelona,19.0,17.0,18.0,21.0,15.0,18.0,18.0,16.0,16.0,...,27.0,31.0,30.0,28.0,32.0,27.0,30.0,29.0,28.0,28.0
2,Atletico de Madrid,17.0,14.0,20.0,18.0,11.0,18.0,19.0,16.0,14.0,...,20.0,13.0,17.0,15.0,23.0,28.0,23.0,28.0,23.0,23.0
3,Valencia,18.0,19.0,12.0,13.0,12.0,12.0,13.0,16.0,14.0,...,18.0,21.0,21.0,17.0,19.0,13.0,22.0,11.0,13.0,22.0
4,Sevilla,13.0,9.0,16.0,15.0,22.0,13.0,11.0,13.0,12.0,...,21.0,19.0,17.0,13.0,14.0,18.0,23.0,14.0,21.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Ontinyent,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,Universidad de Las Palmas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,Lorca FC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
113,Moscardo,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# Linear refression model to check score for the next season

from scipy import stats

def myfunc(x):
  return slope * x + intercept

x = col
lst=[]

for i in range(0,115):
  y = list(df.loc[i,])
  y.pop(0)

  slope, intercept, r, p, std_err = stats.linregress(x, y)
  speed = myfunc(2018)
  lst.append(round(speed,1))

print(lst)


[27.6, 28.1, 20.2, 19.0, 18.2, 14.4, 14.8, 15.6, 14.9, 14.3, 15.5, 14.7, 15.0, 12.7, 11.5, 14.4, 13.8, 11.8, 11.0, 7.2, 13.3, 4.1, 10.3, 6.8, 6.2, 20.3, 2.7, 6.6, 12.2, 0.0, 15.2, 10.4, 13.1, 4.4, 13.0, 7.1, 16.8, 1.1, -3.6, 15.0, 2.9, 9.8, 14.4, 14.1, 1.4, 8.3, 5.2, 8.5, 1.7, 11.5, -4.3, 4.1, 1.4, 4.2, 3.3, 9.5, 0.9, 8.2, 3.5, 2.7, 2.1, 0.8, -3.2, 4.4, 6.4, 1.3, 0.5, -2.1, 5.3, 0.8, -1.4, 1.0, 4.8, -0.2, 3.0, -1.3, -1.4, 0.3, -0.8, -0.6, 2.9, -0.3, 0.9, -0.8, 2.8, 2.5, -0.2, 1.5, 1.5, 2.1, 1.8, 2.0, -0.9, 0.5, 0.2, -0.6, 0.1, 0.7, 0.5, 0.4, 0.9, -0.3, -0.2, 0.6, -0.0, 0.1, 0.6, 0.5, -0.0, 0.5, -0.3, 0.3, 0.7, -0.2, 0.0]


In [36]:
# Winner predicting using linear regression

ind = lst.index(max(lst))
print(df.team[ind])

# winner was predicted correctly

Barcelona


In [37]:
# Teams with regression score less than 1, this shows teams which are predicted not win a single game
# These teams might be relegated

reg = [df.team[lst.index(i)] for i in lst if i<1]


In [38]:
teams_2018_19 = ["Alaves","Athletic Bilbao","Atletico Madrid","Barcelona","Celta Vigo","Deportivo La Coruña","Eibar","Espanyol","Getafe","Girona","Universidad de Las Palmas",
                 "Leganes","Levante","CD Malaga","Real Betis","Real Madrid","Real Sociedad","Sevilla","Valencia","Valladolid","Villarreal"]


In [39]:
# teams relegated in 2018_19 season

for i in reg:
  if i in teams_2018_19:
    print(i)

# 1 out of 3 teams were predicted

CD Malaga
